In [1]:
import pandas as pd

In [9]:
data01 = pd.read_csv('/home/suneel/Desktop/chess-engine/Data/Data Lake/Chess_1.csv')
data02 = pd.read_csv('/home/suneel/Desktop/chess-engine/Data/Data Lake/Chess_2.csv')
data03 = pd.concat([data01,data02],axis=0)
data03

,FEN,Evaluation
0,r2q1rk1/pb3pbp/1ppp1np1/2nPp3/4P3/2P1B2P/PPBNN...,16
1,r7/ppQ1n2p/2n1k1p1/1N1p1q2/1P1P4/2P3P1/P7/2K5 ...,-929
2,r1bq1rk1/ppp1bppp/2n2n2/3Pp3/2B5/2PP1N1P/PP3PP...,4
3,6k1/R2b2r1/8/7R/2PPP3/8/7P/5K2 b - - 0 42,546
4,r4r2/pp4pp/k1nQ1q2/5P2/2B5/8/P5PP/1R3RK1 b - -...,684
...,...,...
1980494,rnb1k2r/pp3ppp/4pn2/3pB3/8/P2PqN2/2PKPbPP/RN1Q...,-402
1980495,6R1/pq1k1p2/1p1pp3/8/8/1P5r/P1PPQP1P/R5K1 w - ...,641
1980496,1r3rk1/pp4p1/1qn4p/4pp2/8/2NPPNP1/PP1Q1KbP/R7 ...,-357
1980497,rnb1k1r1/pp3pBp/2pqp1p1/3P4/8/P1N2N2/1PP2PPP/R...,686


In [10]:
data03 = data03.reset_index()

In [11]:
data04 = data03.copy()
mean = data04['Evaluation'].mean()
std = data04['Evaluation'].std()
data04['Evaluation_std'] = (data04['Evaluation'] - mean) / std
print("mean : ",mean)
print("std : ",std)
data04

mean :  13.409747727645131
std :  588.6268786929631


,index,FEN,Evaluation,Evaluation_std
0,0,r2q1rk1/pb3pbp/1ppp1np1/2nPp3/4P3/2P1B2P/PPBNN...,16,0.004400
1,1,r7/ppQ1n2p/2n1k1p1/1N1p1q2/1P1P4/2P3P1/P7/2K5 ...,-929,-1.601031
2,2,r1bq1rk1/ppp1bppp/2n2n2/3Pp3/2B5/2PP1N1P/PP3PP...,4,-0.015986
3,3,6k1/R2b2r1/8/7R/2PPP3/8/7P/5K2 b - - 0 42,546,0.904801
4,4,r4r2/pp4pp/k1nQ1q2/5P2/2B5/8/P5PP/1R3RK1 b - -...,684,1.139245
...,...,...,...,...
3960754,1980494,rnb1k2r/pp3ppp/4pn2/3pB3/8/P2PqN2/2PKPbPP/RN1Q...,-402,-0.705727
3960755,1980495,6R1/pq1k1p2/1p1pp3/8/8/1P5r/P1PPQP1P/R5K1 w - ...,641,1.066194
3960756,1980496,1r3rk1/pp4p1/1qn4p/4pp2/8/2NPPNP1/PP1Q1KbP/R7 ...,-357,-0.629278
3960757,1980497,rnb1k1r1/pp3pBp/2pqp1p1/3P4/8/P1N2N2/1PP2PPP/R...,686,1.142643


In [20]:
import numpy as np

def encoding(fen):
    '''
    12 planes for each piece type (K, k, P, p, B, b, N, n, R, r, Q, q)
    1 plane for empty spaces
    1 plane for active color (1 for white, -1 for black)
    4 planes for castling rights (K, Q, k, q)
    1 plane for en passant target square (1 at the target square, 0 elsewhere)
    1 plane for halfmove clock
    1 plane for fullmove number
    '''
    # Piece indexing
    piece_indexes = {
        'K': 1, 'k': 2, 'P': 3, 'p': 4, 'B': 5, 'b': 6, 'N': 7, 'n': 8, 
        'R': 9, 'r': 10, 'Q': 11, 'q': 12
    }

    # Parse the FEN string
    position, active, castling, enpassant, halfmove, fullmove = fen.split(' ')

    # Initialize the encoded array with zeros
    encoded = np.zeros((21, 8, 8))  # Updated to 21 channels

    # Split the position into ranks (8 rows)
    ranks = position.split('/')

    # Encode the pieces and empty squares
    for i in range(8):
        rank = ranks[i]
        ind = 0
        for char in rank:
            if char.isdigit():  # Empty squares
                encoded[0, i, ind:ind+int(char)] = 1
                ind += int(char)
            else:
                encoded[piece_indexes[char], i, ind] = 1
                ind += 1

    # Encode the active color (1 for white, -1 for black)
    encoded[13] = 1 if active == 'w' else -1

    # Encode castling rights (K, Q, k, q)
    castling_map = {'K': 14, 'Q': 15, 'k': 16, 'q': 17}
    for right in castling:
        if right in castling_map:
            encoded[castling_map[right]] = 1  # Set the entire plane to 1

    # Encode en passant (1 at the en passant square, 0 elsewhere)
    if enpassant != '-':
        file, rank = ord(enpassant[0]) - ord('a'), 8 - int(enpassant[1])
        encoded[18, rank, file] = 1

    # Encode halfmove and fullmove counters
    encoded[19] = int(halfmove)
    encoded[20] = int(fullmove)

    return encoded


In [21]:
fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
#print(fen.split(' '))
print(encoding(fen))

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]]


In [22]:
import torch
import torch.nn as nn
t1 = torch.tensor(encoding(fen))
print(t1.shape)

torch.Size([21, 8, 8])


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BaseModel2(nn.Module):
    def __init__(self):
        super(BaseModel2, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(21, 256, kernel_size=3, padding='same')
        self.norm1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=3, padding='same')
        self.norm2 = nn.BatchNorm2d(256)

        # Fully connected layers
        self.hiddenlayer1 = nn.Linear(256 * 8 * 8,256)  
        self.norm3 = nn.BatchNorm1d(256)
        self.hiddenlayer2 = nn.Linear(256, 512)
        self.norm4 = nn.BatchNorm1d(512)
        self.hiddenlayer3 = nn.Linear(512, 256)
        self.norm5 = nn.BatchNorm1d(256)
        self.output = nn.Linear(256, 1)

    def forward(self, x):
        x = F.leaky_relu(self.norm1(self.conv1(x)))
        x = F.leaky_relu(self.norm2(self.conv2(x)))

        x = x.view(x.size(0), -1)  
        x = F.leaky_relu(self.norm3(self.hiddenlayer1(x)))
        x = F.leaky_relu(self.norm4(self.hiddenlayer2(x)))
        x = F.leaky_relu(self.norm5(self.hiddenlayer3(x)))

        return self.output(x)


In [31]:
from torch.utils.data import Dataset,DataLoader
import pandas as pd

class Chessloader(Dataset) :
    def __init__(self,df) :
        self.df = df
    def __len__(self) :
        return len(self.df)
    def __getitem__(self,ind) :
        row = self.df.iloc[ind]
        tensor = torch.tensor(encoding(row['FEN']),dtype=torch.float32).squeeze()
        target = torch.tensor(row['Evaluation_std'], dtype=torch.float32)
        return tensor,target

In [60]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(data04.sample(2500000,random_state=42),test_size=0.20,random_state=42)
train_data = train_data[train_data['Evaluation_std'].between(-3,3)]

In [61]:
train_data

,index,FEN,Evaluation,Evaluation_std
2349987,369727,1B6/8/4k3/8/5p2/5P2/5K2/8 w - - 5 54,539,0.892909
1768825,1768825,rn1qkb1r/pppbpppp/3p4/8/2B1n3/3P1N2/PPP2PPP/RN...,-83,-0.163788
2380644,400384,r2qkb1r/1p1nnpp1/p1p1p2p/3pPb2/2PP4/5N1P/PP2BP...,1,-0.021083
252723,252723,5N1B/1p1k1P2/3p1R2/2p1p3/P5Pp/6nK/4r3/R7 b - -...,665,1.106967
2661834,681574,2rk1b1r/3b1Qp1/3p1p1p/1p1BpP2/pP1pP3/3P2RN/1q4...,-163,-0.299697
...,...,...,...,...
3629622,1649362,r2q1rk1/pppbbppp/2n5/3p4/3PnP2/3BP3/PP1N2PP/R1...,56,0.072355
2106623,126363,r6k/ppp4p/4bb2/4p2p/2P1Pp2/1P1r1N1P/P5PK/R1B3R...,-213,-0.384641
2429332,449072,rnbqkb1r/3p1ppp/p3pn2/2pP4/2B1P3/2N5/PPP2PPP/R...,245,0.393442
631950,631950,r3k2B/p1qnb2p/6p1/5p2/3p4/1P2p3/P3Q1PP/RN3RK1 ...,336,0.548039


In [62]:
train_set = Chessloader(train_data.sample(10000))
test_set = Chessloader(test_data.sample(1000))
train_loader = DataLoader(train_set,batch_size=32,shuffle=False)
test_loader  = DataLoader(test_set,batch_size=32,shuffle=False)

In [66]:
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import os
import math

class CustomLoss(nn.Module):
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha
        self.mse = nn.MSELoss()
    
    def forward(self, pred, target):
        mse_loss = self.mse(pred, target)
        # Calculate scaling factor based on target values
        scale = torch.log(math.e + self.alpha * torch.abs(target))
        return (mse_loss * scale).mean()

def train_model(model, train_loader, test_loader, epochs=40, lr=0.0005, alpha=1.0, checkpoint_dir='/home/suneel/Desktop/chess-engine/models/basemodel2/alpha1'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Initialize losses
    train_criterion = CustomLoss(alpha=alpha).to(device)
    test_criterion = nn.MSELoss()  # Keep regular MSE for testing
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    
    # Track best model
    best_test_loss = float('inf')
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        train_pbar = tqdm(train_loader, desc=f'Epoch [{epoch+1}/{epochs}] Training')
        
        for inputs, targets in train_pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            targets = targets.view(-1, 1)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = train_criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            train_pbar.set_postfix({'train_loss': f'{loss.item():.4f}'})
            
        avg_train_loss = running_loss / len(train_loader)
        
        # Evaluation phase
        model.eval()
        test_loss = 0.0
        test_pbar = tqdm(test_loader, desc=f'Epoch [{epoch+1}/{epochs}] Testing')
        
        # Keep track of last 5 batches
        last_5_predictions = []
        last_5_actuals = []
        
        with torch.no_grad():
            for inputs, targets in test_pbar:
                inputs, targets = inputs.to(device), targets.to(device)
                targets = targets.view(-1, 1)
                outputs = model(inputs)
                loss = test_criterion(outputs, targets)
                test_loss += loss.item()
                test_pbar.set_postfix({'test_loss': f'{loss.item():.4f}'})
                
                # Store predictions and actuals for this batch
                last_5_predictions = outputs.cpu().numpy()[-5:]
                last_5_actuals = targets.cpu().numpy()[-5:]
        
        avg_test_loss = test_loss / len(test_loader)
        
        # Save checkpoint if this is the best model so far
        if avg_test_loss < best_test_loss:
            best_test_loss = avg_test_loss
            checkpoint_path = os.path.join(checkpoint_dir, f'best_model_epoch_{epoch+1}.pth')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss,
                'test_loss': avg_test_loss,
            }, checkpoint_path)
            print(f"\nSaved new best model with test loss: {avg_test_loss:.4f}")
        
        print(f"\nEpoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")
        print("\nLast 5 test samples for this epoch:")
        print("Predictions vs Actuals:")
        for i, (pred, actual) in enumerate(zip(last_5_predictions, last_5_actuals)):
            print(f"Sample {i+1}: Predicted = {pred[0]:.4f}, Actual = {actual[0]:.4f}")
        print("-" * 50)
    
    return model

# Function to load the best model


In [67]:
def load_best_model(model, checkpoint_dir='checkpoints'):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]
    if not checkpoints:
        return None, None
    
    # Get the checkpoint with the lowest test loss from the filename
    best_checkpoint = sorted(checkpoints, key=lambda x: float(torch.load(os.path.join(checkpoint_dir, x))['test_loss']))[0]
    checkpoint_path = os.path.join(checkpoint_dir, best_checkpoint)
    
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return model, checkpoint

In [68]:
import torch.nn as nn
import torch

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

    elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
        nn.init.ones_(m.weight)  # Set γ (scale) to 1
        nn.init.zeros_(m.bias)   # Set β (shift) to 0

# Example usage:
# model = BaseModel2()
# model.apply(init_weights)


In [69]:
model = BaseModel2()
model.apply(init_weights)
print(model)

BaseModel2(
  (conv1): Conv2d(21, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (norm1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (norm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hiddenlayer1): Linear(in_features=16384, out_features=256, bias=True)
  (norm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hiddenlayer2): Linear(in_features=256, out_features=512, bias=True)
  (norm4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hiddenlayer3): Linear(in_features=512, out_features=256, bias=True)
  (norm5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output): Linear(in_features=256, out_features=1, bias=True)
)


In [70]:
train_model(model,train_loader,test_loader)

Epoch [1/40] Testing: 100%|██████████| 32/32 [00:00<00:00, 46.05it/s, test_loss=0.5467]



Saved new best model with test loss: 1.0038

Epoch [1/40], Train Loss: 8.7029, Test Loss: 1.0038

Last 5 test samples for this epoch:
Predictions vs Actuals:
Sample 1: Predicted = -0.1821, Actual = -0.0839
Sample 2: Predicted = 0.9728, Actual = -0.0279
Sample 3: Predicted = -0.4842, Actual = 0.1760
Sample 4: Predicted = 0.1676, Actual = -0.8060
Sample 5: Predicted = -0.3613, Actual = -0.3150
--------------------------------------------------


Epoch [2/40] Testing: 100%|██████████| 32/32 [00:01<00:00, 30.89it/s, test_loss=0.5063]



Epoch [2/40], Train Loss: 1.1019, Test Loss: 1.0091

Last 5 test samples for this epoch:
Predictions vs Actuals:
Sample 1: Predicted = -0.1048, Actual = -0.0839
Sample 2: Predicted = 0.9912, Actual = -0.0279
Sample 3: Predicted = -0.5257, Actual = 0.1760
Sample 4: Predicted = 0.1443, Actual = -0.8060
Sample 5: Predicted = -0.0832, Actual = -0.3150
--------------------------------------------------


Epoch [3/40] Testing: 100%|██████████| 32/32 [00:00<00:00, 47.39it/s, test_loss=0.5975]


...


Epoch [25/40], Train Loss: 0.3382, Test Loss: 1.6731

Last 5 test samples for this epoch:
Predictions vs Actuals:
Sample 1: Predicted = 1.5899, Actual = -0.0839
Sample 2: Predicted = -0.1819, Actual = -0.0279
Sample 3: Predicted = 0.5370, Actual = 0.1760
Sample 4: Predicted = 0.5540, Actual = -0.8060
Sample 5: Predicted = 0.8250, Actual = -0.3150
--------------------------------------------------


Epoch [26/40] Testing: 100%|██████████| 32/32 [00:01<00:00, 30.40it/s, test_loss=3.2159]



Epoch [26/40], Train Loss: 0.3482, Test Loss: 1.9196

Last 5 test samples for this epoch:
Predictions vs Actuals:
Sample 1: Predicted = -1.3474, Actual = -0.0839
Sample 2: Predicted = -0.2859, Actual = -0.0279
Sample 3: Predicted = 0.4336, Actual = 0.1760
Sample 4: Predicted = -0.0063, Actual = -0.8060
Sample 5: Predicted = -1.4962, Actual = -0.3150
--------------------------------------------------


Epoch [27/40] Training:  31%|███       | 97/313 [00:09<00:20, 10.38it/s, train_loss=0.4222]


KeyboardInterrupt: 